In [18]:
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.schema.runnable import RunnablePassthrough

In [3]:
class AddToShoppingCart(BaseModel):
    """添加商品到购物车"""
    name: str
    """商品名称"""
    price: float
    """商品价格"""

class RemoveFromShoppingCart(BaseModel):
    """从购物车中移除商品"""
    name: str
    """商品名称"""

class ReviewShoppingCart(BaseModel):
    """查看购物车清单"""

class Checkout(BaseModel):
    """购物车结算"""
    

In [40]:
template = """根据客户的问题选择对应的方法去完成客户的需求，方法需要的参数可以从历史对话中检索出来。如果信息不全，参数可以用None替代。不要凭空构造数值
history:
{history}

request:
{request}
"""

prompt = PromptTemplate.from_template(template)

In [33]:
history = ChatPromptTemplate.from_messages([
    ("human", "我想买一本 Python 入门的书籍"),
    ("ai", "我这里有《Python in Action》，这本书比较适合入门。也不贵，只需要 51.2 元")
])
history

ChatPromptTemplate(input_variables=[], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='我想买一本 Python 入门的书籍')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='我这里有《Python in Action》，这本书比较适合入门。也不贵，只需要 51.2 元'))])

In [20]:
history

'Human: 我想买一本 Python 入门的书籍\nAI: 我这里有《Python in Action》，这本书比较适合入门。也不贵，只需要 51.2 元'

In [34]:
RunnablePassthrough.assign(history=history)

RunnableAssign(mapper={
  history: ChatPromptTemplate(input_variables=[], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='我想买一本 Python 入门的书籍')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='我这里有《Python in Action》，这本书比较适合入门。也不贵，只需要 51.2 元'))])
})

In [42]:
_add = convert_pydantic_to_openai_function(AddToShoppingCart)
_remove = convert_pydantic_to_openai_function(RemoveFromShoppingCart)
_review = convert_pydantic_to_openai_function(ReviewShoppingCart)
_checkout = convert_pydantic_to_openai_function(Checkout)
llm = ChatOpenAI(temperature=0).bind(functions=[_add, _remove, _review, _checkout])
chain = RunnablePassthrough.assign(history=history) | prompt | llm
chain.invoke({"request": "好，我买这本"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'AddToShoppingCart', 'arguments': '{\n  "name": "Python in Action",\n  "price": 51.2\n}'}})

In [46]:
r = chain.invoke({"request": "想了想，那本 Python in action还是不买了，你帮我取消吧"})
r

AIMessage(content='', additional_kwargs={'function_call': {'name': 'RemoveFromShoppingCart', 'arguments': '{\n  "name": "Python in Action"\n}'}})

In [45]:
chain.invoke({"request": "列一下我一共买了那几本书"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ReviewShoppingCart', 'arguments': '{}'}})

In [44]:
chain.invoke({"request": "帮我结账"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'Checkout', 'arguments': '{}'}})

In [47]:
type(r)

langchain.schema.messages.AIMessage

In [52]:
import json
params = r.additional_kwargs["function_call"]["arguments"]
json.loads(params)

{'name': 'Python in Action'}